In [16]:
from pathlib import Path
import sys
project_dir = Path("__file__").resolve().parents[2]
sys.path.insert(0, '{}/temporal_granularity/'.format(project_dir))

import logging

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import pairwise_distances_argmin_min

from sklearn.cluster import KMeans


In [11]:
pv_original = pd.read_csv(
        '{}/data/processed/resources/pv_processed.csv'.format(project_dir))
onshore_original = pd.read_csv(
        '{}/data/processed/resources/onshore_processed.csv'.format(project_dir))
offshore_original = pd.read_csv(
        '{}/data/processed/resources/offshore_processed.csv'.format(project_dir))
load_original = pd.read_csv(
        '{}/data/processed/demand/load_NG/load_processed_normalised.csv'.format(project_dir))

total_data = [pv_original, onshore_original, offshore_original, load_original]

In [15]:
for data in total_data:
    
    data = data.drop('Unnamed: 0', axis=1)

    data.set_index('datetime', inplace=True)
    data.index = pd.to_datetime(data.index)

    data['date'] = data.index.date
    data['hour'] = data.index.hour
    data['year'] = data.index.year
    
    data = data.pivot(
            index='date', columns='hour', values='capacity_factor')
    data = data.dropna()
    
    k_means = KMeans(n_clusters=8)
    k_means.fit(data)
    
    y_kmeans = k_means.predict(data)
    
    y_kmeans_df = pd.DataFrame(y_kmeans)
    y_kmeans_df = y_kmeans_df.rename(columns={y_kmeans_df.columns[0]: 'cluster'})
    
    data_w_kmeans = data.copy()
    data_w_kmeans['cluster'] = y_kmeans

    closest_data_points, _ = pairwise_distances_argmin_min(k_means.cluster_centers_, each_day)
    
    
    